# Notebook for getting nodule_probability_vector for each region in Kaggle-Dataset image

This notebook takes data from Kaggle-data and creates probabilty of nodule presence in each region of the image

In [1]:
import dicom # for reading dicom files
from scipy import ndimage as nd
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import numpy as np
import sklearn

data_dir = './data/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('./stage1_labels.csv', index_col=0)


In [2]:
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]


In [3]:
import matplotlib.pyplot as plt
from keras.models import load_model

for patient in patients:
    print(patient)
    label = labels_df.get_value(patient, 'cancer')
    
    path = data_dir + patient
    
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    num = len(slices)/36
    num_int = int(num)
    last_slice = 36*num_int
    
    
    
    slices = np.array(slices[:last_slice])
    
    temp_arr = [crop_center(slices[i].pixel_array, 504, 504) for i in range(len(slices))]
    temp_arr = np.array(temp_arr)
    print(temp_arr.shape)
    zoomFactors = [504/float(a) for a in temp_arr.shape]

    temp_arr = nd.interpolation.zoom(temp_arr, zoom=zoomFactors) 
    print(temp_arr.shape)
    new_arr =  np.split(temp_arr, 14)
    
    
    
    all_splits = []
    
    model = load_model('10_my_model.h5')

    all_elems = []
    for item in new_arr:

        
        nd_arr = np.split(item,14,1)
        nd_arr = np.array(nd_arr)
        
        for it in nd_arr:
            
            nd_new = np.split(it,14, 2)
            nd_new = np.array(nd_new)
            for elem in nd_new:
                
                elem = elem.reshape(1,36,36,36)
                all_elems.append(elem)
    all_elems = np.array(all_elems)
    new_tuple = model.predict_proba(all_elems, batch_size = 32)
    print(type(new_tuple[0][1]))
    print(new_tuple)


Using TensorFlow backend.


00cba091fa4ad62cc3200a657aeb957e


KeyError: '00cba091fa4ad62cc3200a657aeb957e'